In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *
from train import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
# compute the best hyperparameters for regularized optimization
def best_hyperparameters_accuracy(y, tx, degrees, lambdas, k_fold, seed=1):
    # for each degree, store the best lambda and the respective loss
    ###losses = []
    lambdas_best = []
    accuracies = []
    
    # build k indices for k-fold
    k_indices = build_k_indices(y, k_fold, seed)
    
    # compute cross validation with all lambdas for each degree
    for degree in degrees:
        
        # store the loss, respective to the lambdas
        ###losses_test = []
        accuracies_lambda = []
        
        # compute cross validation for each lambda of the specific degree
        for lambda_ in lambdas:
        
            # to compute the total loss of each lambda by storing the loss for each iteration 
            # of the k-fold and computing the mean
            ###losses_test_tmp = []
            accuracies_tmp = []
        
            # compute loss for each iteration of the k_fold
            for k in range(k_fold):
                weigths, loss_train, loss_test = cross_validation(y, tx, k_indices, k, lambda_, degree)
                ###losses_test_tmp.append(loss_test)
                y_pred = predict_labels(weights, build_poly(tx, degree))
                accuracies_tmp.append(performance_measure(y_pred, y))
            
            #compute the loss for the specific lambda by taking the mean of the losses of each iteration of the k-fold
            accuracies_lambda.append(np.mean(accuracies_tmp))
        
        # find the optimal lambda hyperparameter by getting the minimum loss for each degree
        best_lambda_index = np.argmax(accuracies_lambda)
        lambdas_best.append(lambdas[best_lambda_index])
        accuracies.append(accuracies_lambda[best_lambda_index])
        
    # find the optimal degree hyperparameter by getting the minimum loss
    best_degree_index = np.argmax(accuracies)
    
    # compute the optimal hyperparameters
    opt_degree = degrees[best_degree_index]
    opt_lambda = lambdas_best[best_degree_index]
    
    
    return opt_degree, opt_lambda

In [7]:
tX = np.where(tX==-999, np.NaN, tX)
best_weights, best_degree = train_models(y, tX, np.arange(2, 8), np.logspace(-4, 0, 30), 4)

In [8]:
best_degree

[7, 6, 7, 7, 6, 7]

## Generate predictions and save ouput in csv format for submission:

In [9]:
tX_test = np.where(tX_test==-999, np.NaN, tX_test)

In [10]:
indices_test_group = group_indices(tX_test)

In [11]:
y_pred = np.zeros(tX_test.shape[0])
for i, indice_test_group in enumerate(indices_test_group):
    y_pred_subset = predict_labels(best_weights[i], build_poly(drop_na_columns(tX_test[indice_test_group]), best_degree[i]))
    y_pred[indice_test_group] = y_pred_subset
    
    
y_pred

array([-1., -1., -1., ...,  1., -1., -1.])

In [12]:
OUTPUT_PATH = '../data/sample-submission_test.csv' 
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)